In [38]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import boto3
import numpy
import openslide
from PIL import Image
import sklearn
from skimage import data, io, filters
import os
from pathLib import Path

In [40]:
s3 = boto3.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

aws-cloudtrail-logs-432722299252-98c79c6c
digpath-data
sagemaker-studio-432722299252-gmvals4z90t
sagemaker-us-east-1-432722299252


In [ ]:
bucket = s3.Bucket("digpath-data")
for x, idx in enumerate(bucket.objects.all()):
    if 'Standard' in x.key:
            with open('test-' + idx + '.txt', 'wb') as data:
                s3.download_fileobj('digipath-data', key , data)

In [3]:
from ..python_wsi_preprocessing.deephistopath.wsi import slide
from ..python_wsi_preprocessing.deephistopath.wsi import filter
from ..python_wsi_preprocessing.deephistopath.wsi import tiles
from ..python_wsi_preprocessing.deephistopath.wsi import util


ValueError: attempted relative import beyond top-level package